<a href="https://colab.research.google.com/github/maylovesart/AI-for-Media-Exam/blob/main/UK_Black_Icons_GANs_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set TensorFlow to use GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Load and Preprocess Images
TARGET_SIZE = (256, 256)

def preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, TARGET_SIZE)
    img = img / 127.5 - 1  # Normalize to [-1, 1]
    return img

def load_and_preprocess_images(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(directory, filename)
            img = preprocess_image(img_path)
            images.append(img)
    return tf.stack(images)

# Load images from the specified directories
content_images = load_and_preprocess_images("/content/Diane Abbott")
style_images = load_and_preprocess_images("/content/Richard Stone")

print(f"Loaded {len(content_images)} content images and {len(style_images)} style images.")

# Data Augmentation
def augment_image(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image

# GAN Model Definition
def make_generator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16 * 16 * 256, use_bias=False, input_shape=(100,)),
        tf.keras.layers.Reshape((16, 16, 256)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(128, 5, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(64, 5, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(32, 5, strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Conv2DTranspose(3, 5, strides=2, padding='same', activation='tanh')
    ])
    return model

def make_discriminator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, 5, strides=2, padding='same', input_shape=[256, 256, 3]),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, 5, strides=2, padding='same'),
        tf.keras.layers.LeakyReLU(alpha=0.2),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])
    return model

# Loss Functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output) * 0.9, real_output)  # Label smoothing
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers with learning rate scheduling
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.96
)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)

# Training Step
@tf.function
def train_step(images):
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Generate and Save Images
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, :] * 0.5 + 0.5)  # Denormalize
        plt.axis('off')

    plt.savefig(f'/content/generated_image_epoch_{epoch}.png')
    plt.close()

# Training Loop
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        # Print updates every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Time: {time.time()-start:.2f} sec")
            print(f"Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}")

        # Generate and save images every 20 epochs
        if (epoch + 1) % 20 == 0:
            generate_and_save_images(generator, epoch + 1, seed)

    # Generate a final set of images
    generate_and_save_images(generator, epochs, seed)

# Prepare the dataset
BATCH_SIZE = 32  # Increased batch size
EPOCHS = 500  # Increased number of epochs

# Combine content and style images
combined_images = tf.concat([content_images, style_images], axis=0)
dataset = tf.data.Dataset.from_tensor_slices(combined_images)
dataset = dataset.map(augment_image)  # Apply data augmentation
dataset = dataset.shuffle(1000).batch(BATCH_SIZE, drop_remainder=True)

# Initialize models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Create a seed for image generation
seed = tf.random.normal([16, 100])

# Start training
print("Starting training for 500 epochs...")
train(dataset, EPOCHS)
print("Training complete.")

# Ethical considerations and LLM disclaimer
print("\nEthical Considerations:")
print("This project involves generating images based on real individuals.")
print("It's important to consider the ethical implications of creating and using such images.")
print("The generated images should not be used for misrepresentation or without proper context.")

print("\nLLM Disclaimer:")
print("Large Language Models were used for code structuring and debugging assistance.")
print("All implementation decisions and parameter tuning were made by the human developer.")

Num GPUs Available:  1
Loaded 12 content images and 20 style images.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Starting training for 500 epochs...
Epoch 10/500, Time: 0.30 sec
Generator Loss: 3.2190, Discriminator Loss: 0.4077
Epoch 20/500, Time: 0.31 sec
Generator Loss: 3.1055, Discriminator Loss: 0.5042
Epoch 30/500, Time: 0.33 sec
Generator Loss: 3.3736, Discriminator Loss: 0.3832
Epoch 40/500, Time: 0.33 sec
Generator Loss: 3.8142, Discriminator Loss: 0.3739
Epoch 50/500, Time: 0.34 sec
Generator Loss: 3.7973, Discriminator Loss: 0.3712
Epoch 60/500, Time: 0.33 sec
Generator Loss: 3.9994, Discriminator Loss: 0.4116
Epoch 70/500, Time: 0.32 sec
Generator Loss: 2.8246, Discriminator Loss: 0.4157
Epoch 80/500, Time: 0.32 sec
Generator Loss: 2.4196, Discriminator Loss: 0.4561
Epoch 90/500, Time: 0.30 sec
Generator Loss: 2.8741, Discriminator Loss: 0.4886
Epoch 100/500, Time: 0.29 sec
Generator Loss: 2.6603, Discriminator Loss: 0.4834
Epoch 110/500, Time: 0.29 sec
Generator Loss: 0.6460, Discriminator Loss: 1.3016
Epoch 120/500, Time: 0.29 sec
Generator Loss: 2.5519, Discriminator Loss: 0.4825
E